In [ ]:
# Install required packages
!pip install tensorflow==2.12.0
!pip install pandas
!pip install gdown

INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 111.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 119.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 MB 9.6 MB/s eta 0:00:00
  At

Question 1

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, GRU, SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
# Load Hindi data
def load_hindi_data():
    train_path = "/content/hi.translit.sampled.train.tsv"
    dev_path = "/content/hi.translit.sampled.dev.tsv"
    test_path = "/content/hi.translit.sampled.test.tsv"

    train_data = pd.read_csv(train_path, sep='\t', header=None,
                           names=['devanagari', 'latin', 'count'])
    dev_data = pd.read_csv(dev_path, sep='\t', header=None,
                         names=['devanagari', 'latin', 'count'])
    test_data = pd.read_csv(test_path, sep='\t', header=None,
                          names=['devanagari', 'latin', 'count'])

    return train_data, dev_data, test_data

train_data, dev_data, test_data = load_hindi_data()

# Show some samples
print("Sample training data:")
print(train_data.head())

Sample training data:
  devanagari     latin  count
0         अं        an      3
1    अंकगणित  ankganit      3
2       अंकल     uncle      4
3      अंकुर     ankur      4
4     अंकुरण   ankuran      3


Preprocess the data

In [ ]:
def preprocess_data(train_data, dev_data, test_data, max_sequence_length=20):
    # First, clean the data by removing any rows with NaN values
    train_data = train_data.dropna()
    dev_data = dev_data.dropna()
    test_data = test_data.dropna()

    # Combine all data for vocabulary creation
    all_latin = pd.concat([train_data['latin'], dev_data['latin'], test_data['latin']])
    all_devanagari = pd.concat([train_data['devanagari'], dev_data['devanagari'], test_data['devanagari']])

    # Convert to string type to ensure we don't have any numeric values
    all_latin = all_latin.astype(str)
    all_devanagari = all_devanagari.astype(str)

    # Create character-level tokenizers
    latin_tokenizer = Tokenizer(char_level=True, lower=False)
    latin_tokenizer.fit_on_texts(all_latin)

    devanagari_tokenizer = Tokenizer(char_level=True, lower=False)
    devanagari_tokenizer.fit_on_texts(all_devanagari)

    # Add start and end tokens for decoder sequences
    devanagari_tokenizer.word_index['<start>'] = len(devanagari_tokenizer.word_index) + 1
    devanagari_tokenizer.word_index['<end>'] = len(devanagari_tokenizer.word_index) + 1

    # Convert texts to sequences
    def process_sequences(texts, tokenizer, max_len):
        # Ensure all texts are strings
        texts = [str(text) for text in texts]
        seq = tokenizer.texts_to_sequences(texts)
        seq = pad_sequences(seq, maxlen=max_len, padding='post')
        return seq

    # Process input (Latin) sequences
    X_train = process_sequences(train_data['latin'], latin_tokenizer, max_sequence_length)
    X_dev = process_sequences(dev_data['latin'], latin_tokenizer, max_sequence_length)
    X_test = process_sequences(test_data['latin'], latin_tokenizer, max_sequence_length)

    # Process target (Devanagari) sequences with start/end tokens
    def process_target_sequences(texts, tokenizer, max_len):
        # Ensure all texts are strings
        texts = [str(text) for text in texts]
        seq = tokenizer.texts_to_sequences(texts)
        # Add start and end tokens
        seq = [[tokenizer.word_index['<start>']] + s + [tokenizer.word_index['<end>']] for s in seq]
        seq = pad_sequences(seq, maxlen=max_len+2, padding='post')  # +2 for start/end tokens
        return seq

    y_train = process_target_sequences(train_data['devanagari'], devanagari_tokenizer, max_sequence_length)
    y_dev = process_target_sequences(dev_data['devanagari'], devanagari_tokenizer, max_sequence_length)
    y_test = process_target_sequences(test_data['devanagari'], devanagari_tokenizer, max_sequence_length)

    # Create decoder input (shifted by one) and output data
    decoder_input_train = y_train[:, :-1]
    decoder_output_train = y_train[:, 1:]

    decoder_input_dev = y_dev[:, :-1]
    decoder_output_dev = y_dev[:, 1:]

    decoder_input_test = y_test[:, :-1]
    decoder_output_test = y_test[:, 1:]

    # One-hot encode the output
    def one_hot_encode(sequences, vocab_size):
        return np.array([tf.keras.utils.to_categorical(s, num_classes=vocab_size) for s in sequences])

    vocab_size = len(devanagari_tokenizer.word_index) + 1  # +1 for 0 padding

    decoder_output_train = one_hot_encode(decoder_output_train, vocab_size)
    decoder_output_dev = one_hot_encode(decoder_output_dev, vocab_size)
    decoder_output_test = one_hot_encode(decoder_output_test, vocab_size)

    return (X_train, decoder_input_train, decoder_output_train,
            X_dev, decoder_input_dev, decoder_output_dev,
            X_test, decoder_input_test, decoder_output_test,
            latin_tokenizer, devanagari_tokenizer)

(X_train, decoder_input_train, decoder_output_train,
 X_dev, decoder_input_dev, decoder_output_dev,
 X_test, decoder_input_test, decoder_output_test,
 latin_tokenizer, devanagari_tokenizer) = preprocess_data(train_data, dev_data, test_data)

Build and train the model

In [ ]:
def build_seq2seq_model(input_vocab_size, target_vocab_size, embedding_dim=64,
                       hidden_units=128, cell_type='lstm'):
    # Encoder
    encoder_inputs = Input(shape=(None,))
    encoder_embedding = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)

    # Choose RNN cell
    if cell_type == 'lstm':
        encoder_rnn = LSTM(hidden_units, return_state=True)
        encoder_outputs, state_h, state_c = encoder_rnn(encoder_embedding)
        encoder_states = [state_h, state_c]
    elif cell_type == 'gru':
        encoder_rnn = GRU(hidden_units, return_state=True)
        encoder_outputs, state_h = encoder_rnn(encoder_embedding)
        encoder_states = [state_h]
    else:  # SimpleRNN
        encoder_rnn = SimpleRNN(hidden_units, return_state=True)
        encoder_outputs, state_h = encoder_rnn(encoder_embedding)
        encoder_states = [state_h]

    # Decoder
    decoder_inputs = Input(shape=(None,))
    decoder_embedding = Embedding(target_vocab_size, embedding_dim)(decoder_inputs)

    if cell_type == 'lstm':
        decoder_rnn = LSTM(hidden_units, return_sequences=True, return_state=True)
        decoder_outputs, _, _ = decoder_rnn(decoder_embedding, initial_state=encoder_states)
    elif cell_type == 'gru':
        decoder_rnn = GRU(hidden_units, return_sequences=True, return_state=True)
        decoder_outputs, _ = decoder_rnn(decoder_embedding, initial_state=encoder_states)
    else:
        decoder_rnn = SimpleRNN(hidden_units, return_sequences=True, return_state=True)
        decoder_outputs, _ = decoder_rnn(decoder_embedding, initial_state=encoder_states)

    decoder_dense = Dense(target_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    # Training model
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    # Inference models
    encoder_model = Model(encoder_inputs, encoder_states)

    decoder_state_inputs = [Input(shape=(hidden_units,)) for _ in encoder_states]
    decoder_outputs, *decoder_states = decoder_rnn(
        decoder_embedding, initial_state=decoder_state_inputs)
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model(
        [decoder_inputs] + decoder_state_inputs,
        [decoder_outputs] + decoder_states)

    return model, encoder_model, decoder_model

# Build the model
input_vocab_size = len(latin_tokenizer.word_index) + 1
target_vocab_size = len(devanagari_tokenizer.word_index) + 1

model, encoder_model, decoder_model = build_seq2seq_model(
    input_vocab_size, target_vocab_size,
    embedding_dim=64, hidden_units=128, cell_type='lstm')

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 64)  │      1,728 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 64)  │      4,224 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 128),     │     98,816 │ embedding[0][0]   │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │     98,816 │ embedding_1[0][0… │
│                     │ 128), (None,      │            │ lstm[0][1],       │
│                     │ 128), (None,      │            │ lstm[0][2]        │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 66)  │      8,514 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 212,098 (828.51 KB)

 Trainable params: 212,098 (828.51 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
history = model.fit(
    [X_train, decoder_input_train],
    decoder_output_train,
    batch_size=64,
    epochs=30,
    validation_data=([X_dev, decoder_input_dev], decoder_output_dev),
    verbose=1
)

Epoch 1/30
691/691 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.7084 - loss: 1.2910 - val_accuracy: 0.7643 - val_loss: 0.8593
Epoch 2/30
691/691 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.7631 - loss: 0.8641 - val_accuracy: 0.7876 - val_loss: 0.7434
Epoch 3/30
691/691 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.7955 - loss: 0.7181 - val_accuracy: 0.8392 - val_loss: 0.5507
Epoch 4/30
691/691 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.8445 - loss: 0.5300 - val_accuracy: 0.8764 - val_loss: 0.4120
Epoch 5/30
691/691 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.8813 - loss: 0.3946 - val_accuracy: 0.8999 - val_loss: 0.3268
Epoch 6/30
691/691 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.9044 - loss: 0.3134 - val_accuracy: 0.9146 - val_loss: 0.2823
Epoch 7/30
691/691 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.9199 - loss: 0.2627 - val_accuracy: 0.9243 - val_loss: 0.2477
Epoch 8/30
691/691 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.9282 - loss: 0.2319 - val_accura

In [ ]:
def decode_sequence(input_seq, encoder_model, decoder_model,
                   latin_tokenizer, devanagari_tokenizer, max_length=20):
    # Encode input
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = devanagari_tokenizer.word_index['<start>']

    reverse_target_char_index = {i: char for char, i in devanagari_tokenizer.word_index.items()}

    decoded_sentence = []
    for _ in range(max_length):
        output_tokens, *states_value = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence.append(sampled_char)

        if sampled_char == '<end>':
            break

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

    return ''.join([c for c in decoded_sentence if c not in ['<start>', '<end>']])

# Evaluate on test set
test_loss, test_acc = model.evaluate([X_test, decoder_input_test], decoder_output_test, verbose=0)
print(f"Test Accuracy: {test_acc:.4f}")

# Show some predictions
for i in range(5):
    input_seq = X_test[i:i+1]
    decoded = decode_sequence(
        input_seq, encoder_model, decoder_model,
        latin_tokenizer, devanagari_tokenizer)

    original_input = latin_tokenizer.sequences_to_texts([X_test[i]])[0]
    original_target = devanagari_tokenizer.sequences_to_texts([decoder_input_test[i]])[0]

    print(f"\nSample {i+1}:")
    print(f"Input (Latin): {original_input}")
    print(f"Target (Devanagari): {original_target}")
    print(f"Predicted (Devanagari): {decoded}")

Test Accuracy: 0.9457
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step

Sample 1:
Input (Latin): a n k
Target (Devanagari): अ ं क
Predicted (Devanagari): ऐंक
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step

Sample 2:
Input (Latin): a n k a
Target (Devanagari): अ ं क
Predicted (Devanagari): अंका
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step

Sample 3:
Input (Latin): a n k i t
Target (Devanagari): अ ं क ि त
Predicted (Devanagari): अंकित
1/1 ━━━━━━━━━━━━━━

Question 2